In [ ]:
!pip install tensorflow

In [ ]:
import random 
random.seed(123)
from math import *
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures,MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import train_test_split
import operator as op

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential

In [ ]:
data = pd.read_csv("dataset.txt",sep=";")
data.head()

In [ ]:
missing = ['Sub_metering_3']
for i in missing:
    data.loc[data.loc[:,i].isnull(),i]=data.loc[:,i].mean()

In [ ]:
data['Date'] = data['Date'].astype(str)
data['Time'] = data['Time'].astype(str)
data.replace(['?', 'nan', np.nan], -1, inplace=True) 
num_col= ['Global_active_power', 'Global_reactive_power', 'Voltage', 
           'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
for i in num_col:
    data[i] = pd.to_numeric(data[i])

In [ ]:
# Filling nan and all other illegal values with mean
for i in num_col:
    mean_ = data.loc[:,i].mean()
    data.loc[data.loc[:,i] == -1,i]=mean_

In [ ]:
data.info()

In [ ]:
def convert(x):
    l = list(x.split('/'))
    return l[2]+"-"+l[1]+"-"+l[0]

In [ ]:
data['Date'] = data['Date'].map(lambda x : convert(x))
data.head()

In [ ]:
data['Date_and_Time'] = data['Date'].str.cat(data['Time'],sep = " ")
data = data.drop(['Date','Time'],axis = 1)
data.head()

In [ ]:
data.shape

In [ ]:
data['Date_and_Time'] = pd.to_datetime(data['Date_and_Time'])
data = data.set_index('Date_and_Time').resample('60min').mean()
data.info()
data.head()

In [ ]:
data.shape

In [ ]:
def series_to_supervised(data, in_cnt=1, out_cnt=1, dropnan=True):
    var_cnt = 0
    if type(data) is list :
        var_cnt = 1
    else:
        var_cnt= data.shape[1]
        
    temp_df = pd.DataFrame(data)
    cols, col_names = list(), list()
    for i in range(in_cnt, 0, -1):
        cols.append(temp_df.shift(-i))
        col_names += [('var%d(t-%d)' % (j+1, i)) for j in range(var_cnt)]
    for i in range(0, out_cnt):
        cols.append(temp_df.shift(-i))
        if i==0:
            col_names += [('var%d(t)' % (j+1)) for j in range(var_cnt)]
        else:
            col_names += [('var%d(t+%d)' % (j+1)) for j in range(var_cnt)]
        
        agg = pd.concat(cols, axis=1)
        agg.columns = col_names
        if dropnan:
            agg.dropna(inplace=True)
        drop_col = [x for x in range(8,14)]
        agg.drop(agg.columns[drop_col],axis=1,inplace = True)
        return agg

In [ ]:
transformed_data = series_to_supervised(data, 1, 1)

In [ ]:
transformed_data.head()

In [ ]:
train_time_cnt = 365*24*3
data_values = transformed_data.values
train = data_values[:train_time_cnt, :]
test = data_values[train_time_cnt:, :]
X_train = train[:,:7]
Y_train = train[:,7:]
X_test = test[:,:7]
Y_test = test[:,7:]

In [ ]:
#Visualization of Loss vs epochs

#For training and validation
def validation_vs_training(model_data):
    plt.figure(figsize=(6,4))
    plt.xlabel('epochs')
    plt.ylabel('MSE loss')
    print(type(model_data.history))
    for k,v in model_data.history.items():
        print(k,v)
    l1=[math.sqrt(x) for x in model_data.history['loss']]
    l2=[math.sqrt(x) for x in model_data.history['val_loss']]
    plt.plot(l1,label='Training MSE loss')
    plt.plot(l2,label = 'Validation MSE loss')
    plt.legend(loc='best')
    plt.show()

    
# Learning curves 
def training_loss_curves(model_data):
    pd.DataFrame(model_data.history).plot(figsize=(20, 8))
    plt.title('Learning curves')
    plt.gca().set_ylim(0, 1)
    plt.show()

### 1.Model with one hidden layer
### Optimizer = SGD
### Activation Function = 'Sigmoid'

In [ ]:
def ann_1(no_of_units):
    ann = Sequential(
                [
                    Dense(no_of_units,activation='sigmoid'),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer ='SGD',loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print(Y_test)
    print(pred_values)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))

In [ ]:
ann_1(16)
ann_1(32)
ann_1(64)
ann_1(128)

### 2. Model with two hidden layer
### Optimizer = SGD
### Activation Function = 'Sigmoid'

In [ ]:
def ann_2(no_of_units1,no_of_units2):
    ann= Sequential(
                [
                    Dense(no_of_units1,activation='sigmoid'),
                    Dense(no_of_units2,activation='sigmoid'),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer ='SGD',loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))

In [ ]:
ann_2(16,16)
ann_2(32,32)
ann_2(64,64)
ann_2(128,128)

### 3. Model with three hidden layers 
### Optimizer = SGD
### Activation function = 'Sigmoid'

In [ ]:
def ann_3(no_of_units1, no_of_units2, no_of_units3):
    ann= Sequential(
                [
                    Dense(no_of_units1,activation='sigmoid'),
                    Dense(no_of_units2,activation='sigmoid'),
                    Dense(no_of_units3,activation='sigmoid'),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer ='SGD',loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))
    

In [ ]:
ann_3(16,16,16)
print("###############################################################")
ann_3(32,32,32)
print("###############################################################")
ann_3(64,64,64)
print("###############################################################")
ann_3(128,128,128)

### 4. Model with 1 hidden layer
### Optimizer = SGD
### Activation Function = ReLU

In [ ]:
def ann_4(no_of_units1):
    ann= Sequential(
                [
                    Dense(no_of_units1,activation='relu'),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer ='SGD',loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))

In [ ]:
ann_4(16)
print("####################################################################")
ann_4(32)
print("####################################################################")
ann_4(64)
print("####################################################################")
ann_4(128)


### 5. Model with 2  hidden layers
### Optimizer = SGD
### Activation function = ReLU

In [ ]:

def ann_5(no_of_units1,no_of_units2):
    ann= Sequential(
                [
                    Dense(no_of_units1,activation='relu'),
                    Dense(no_of_units2,activation = 'relu'),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer ='SGD',loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))

In [ ]:
ann_5(16,16)
print("############################################################################")
ann_5(32,32)
print("############################################################################")
ann_5(64,64)
print("############################################################################")
ann_5(128,128)

### 6. Model with 3  hidden layers
### Optimizer = SGD
### Activation function = ReLU

In [ ]:
def ann_6(no_of_units1,no_of_units2,no_of_units3):
    ann= Sequential(
                [
                    Dense(no_of_units1,activation='relu'),
                    Dense(no_of_units2,activation = 'relu'),
                    Dense(no_of_units3,activation='relu'),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer ='SGD',loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))

In [ ]:
ann_6(16,16,16)
print('####################################################################')
ann_6(32,32,32)
print('####################################################################')
ann_6(64,64,64)
print('####################################################################')
ann_6(128,128,128)

### 7.  Model with 1 hidden layer
###     Optimizer = 'Adam'
###     Activation function = 'sigmoid', 'relu'
###     No of units = 16,32,64,128

In [ ]:
def ann_7(no_of_units1,activation_func):
    print("No of units : "+ str(no_of_units1)+",  Activation function : "+activation_func)
    ann= Sequential(
                [
                    Dense(no_of_units1,activation=activation_func),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer ='Adam',loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))

In [ ]:
activation_func_list = ['sigmoid','relu']
no_of_units_list = [16,32,64,128]
for i in activation_func_list:
    for j in no_of_units_list:
        ann_7(j,i)
        print("########################################################################################")

### 8.  Model with 2 hidden layer
###     Optimizer = 'Adam'
###     Activation function = 'sigmoid', 'relu'
###     No of units = 16,32,64,128

In [ ]:
def ann_8(no_of_units1,no_of_units2,activation_func):
    print("No of units : "+ str(no_of_units1)+",  Activation function : "+activation_func)
    ann= Sequential(
                [
                    Dense(no_of_units1,activation=activation_func),
                    Dense(no_of_units2,activation=activation_func),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer ='Adam',loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=40,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))

In [ ]:
ann_8(32,32,'relu')

In [ ]:
activation_func_list = ['sigmoid','relu']
no_of_units_list = [16,32,64,128]
for i in activation_func_list:
    for j in no_of_units_list:
        ann_8(j,j,i)
        print("########################################################################################")

### 9.  Model with 3 hidden layer
###     Optimizer = 'Adam'
###     Activation function = 'sigmoid', 'relu'
###     No of units = 16,32,64,128

In [ ]:
def ann_9(no_of_units1,no_of_units2,no_of_units3,activation_func):
    print("No of units : "+ str(no_of_units1)+",  Activation function : "+activation_func)
    ann= Sequential(
                [
                    Dense(no_of_units1,activation=activation_func),
                    Dense(no_of_units2,activation=activation_func),
                    Dense(no_of_units3,activation=activation_func),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer ='Adam',loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))

In [ ]:
activation_func_list = ['sigmoid','relu']
no_of_units_list = [16,32,64,128]
for i in activation_func_list:
    for j in no_of_units_list:
        ann_9(j,j,j,i)
        print("########################################################################################")

### 10.  Model with 1 hidden layer
###     Optimizer = 'SGD' , 'Adam'
###     Activation function = 'sigmoid', 'relu'
###     No of units = 16,32,64,128
###     Regularization = L2 ( lambda = 0.01)

In [ ]:
def ann_10(optimizer_algo,no_of_units1,activation_func):
    print("Optimizer algorithm : "+ optimizer_algo+" No of units : "+ str(no_of_units1)+",  Activation function : "+activation_func)
    #dense_layer = tf.keras.layers.Dense(units = no_of_units1,activation= activation_func,kernel_regularizer = tf.keras.regularizers.L2(0.01))
    ann= Sequential(
                [
                    Dense(no_of_units1,activation= activation_func,kernel_regularizer = tf.keras.regularizers.L2(0.01)),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer = optimizer_algo,loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))
    

In [ ]:
optimizer_algos = ['SGD','Adam']
activation_func_list = ['sigmoid','relu']
no_of_units_list = [16,32,64,128]
for k in optimizer_algos:
    for i in activation_func_list:
        for j in no_of_units_list:
            ann_10(k,j,i)
            print("########################################################################################")

### 11.  Model with 2 hidden layer
###     Optimizer = 'SGD' , 'Adam'
###     Activation function = 'sigmoid', 'relu'
###     No of units = 16,32,64,128
###     Regularization = L2 ( lambda = 0.01)

In [ ]:
def ann_11(optimizer_algo,no_of_units1,no_of_units2,activation_func):
    print("Optimizer algorithm : "+ optimizer_algo+" No of units : "+ str(no_of_units1)+",  Activation function : "+activation_func)
    #dense_layer = tf.keras.layers.Dense(units = no_of_units1,activation= activation_func,kernel_regularizer = tf.keras.regularizers.L2(0.01))
    ann= Sequential(
                [
                    Dense(no_of_units1,activation= activation_func,kernel_regularizer = tf.keras.regularizers.L2(0.01)),
                    Dense(no_of_units2,activation= activation_func,kernel_regularizer = tf.keras.regularizers.L2(0.01)),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer = optimizer_algo,loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))

In [ ]:
optimizer_algos = ['SGD','Adam']
activation_func_list = ['sigmoid','relu']
no_of_units_list = [16,32,64,128]
for k in optimizer_algos:
    for i in activation_func_list:
        for j in no_of_units_list:
            ann_11(k,j,j,i)
            print("########################################################################################")

### 12.  Model with 3 hidden layer
###     Optimizer = 'SGD' , 'Adam'
###     Activation function = 'sigmoid', 'relu'
###     No of units = 16,32,64,128
###     Regularization = L2 ( lambda = 0.01)

In [ ]:
def ann_12(optimizer_algo,no_of_units1,no_of_units2,no_of_units3,activation_func):
    print("Optimizer algorithm : "+ optimizer_algo+" No of units : "+ str(no_of_units1)+",  Activation function : "+activation_func)
    #dense_layer = tf.keras.layers.Dense(activation= activation_func,kernel_regularizer = tf.keras.regularizers.L2(0.01))
    ann= Sequential(
                [
                    Dense(no_of_units1,activation= activation_func,kernel_regularizer = tf.keras.regularizers.L2(0.0001)),
                    Dense(no_of_units2,activation= activation_func,kernel_regularizer = tf.keras.regularizers.L2(0.0001)),
                    Dense(no_of_units3,activation= activation_func,kernel_regularizer = tf.keras.regularizers.L2(0.0001)),
                    Dense(1,activation = 'linear')
                ])
    ann.compile(optimizer = optimizer_algo,loss='mse',metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])
    model_store = ann.fit(X_train,Y_train,
              epochs=10,
              batch_size=128,
              verbose=0,
              validation_split=0.2)
    ann.summary()
    training_loss_curves(model_store)
    validation_vs_training(model_store)
    res = ann.evaluate(X_test, Y_test, batch_size=128)
    pred_values= ann.predict(X_test)
    print("r2 score :"+str(skm.r2_score(Y_test,pred_values)))

In [ ]:
optimizer_algos = ['SGD','Adam']
activation_func_list = ['sigmoid','relu']
no_of_units_list = [16,32,64,128]
for k in optimizer_algos:
    for i in activation_func_list:
        for j in no_of_units_list:
            ann_12(k,j,j,j,i)
            print("########################################################################################")